# Getting started

To start using the package, you would need: a **model** and an **expression dataset** to integrate with the model. Let's start by exploring how to create an expression dataset.

## Generating an expression profile dataset

To integrate genomics or proteomics data in the model, we need to create a `driven.ExpressionProfile` instance which will handle all the necessary requirements for the purpose. The `ExpressionProfile` class takes a `numpy.ndarray` for the expression data and list-like structures for the **identifiers** (genes/proteins) and **conditons** (media, time, etc.). For this example though, we will build a toy dataset from [doi: 10.3389/fphys.2012.00299](https://doi.org/10.3389/fphys.2012.00299):

In [1]:
import numpy as np
from driven import ExpressionProfile

genes = ["Gene2", "Gene3", "Gene6", "Gene7"]
conditions = ["Exp#1", "Exp#2", "Exp#3"]
expression = np.array(([0.17, 0.20, 0.93],
                       [0.36, 0.83, 0.77],
                       [0.87, 0.65, 0.07],
                       [0.55, 0.49, 0.52]))

model_expression = ExpressionProfile(genes, conditions, expression)

In [2]:
model_expression

,Exp#1,Exp#2,Exp#3
Gene2,0.17,0.20,0.93
Gene3,0.36,0.83,0.77
Gene6,0.87,0.65,0.07
Gene7,0.55,0.49,0.52


## Generating a model

We use [cobrapy](https://github.com/opencobra/cobrapy) to build or load a new model. If you are new to cobrapy, it is recommended to check its [documentation](https://cobrapy.readthedocs.io). Now that we have an expression dataset, we generate the required model like so:

In [3]:
import cobra

model = cobra.Model("Blazier et al 2012")
r_1 = cobra.Reaction("R1")
r_2 = cobra.Reaction("R2")
r_3 = cobra.Reaction("R3")
r_4 = cobra.Reaction("R4")
r_5 = cobra.Reaction("R5")
r_6 = cobra.Reaction("R6")
r_7 = cobra.Reaction("R7")
r_8 = cobra.Reaction("R8")

model.add_reactions([r_1, r_2, r_3, r_4, r_5, r_6, r_7, r_8])

r_1.reaction = "M1_e -> M1"
r_2.reaction = "M1 -> M2"
r_3.reaction = "M2 -> M3"
r_4.reaction = "M3 ->"
r_5.reaction = "M4_e -> M4"
r_6.reaction = "M4 -> M5"
r_7.reaction = "M5 <-> M2"
r_8.reaction = "M5 ->"

r_2.gene_reaction_rule = "Gene2"
r_3.gene_reaction_rule = "Gene3"
r_6.gene_reaction_rule = "Gene6"
r_7.gene_reaction_rule = "Gene7"

EX_M1_e = model.add_boundary(model.metabolites.M1_e, lb=-10.0)
EX_M4_e = model.add_boundary(model.metabolites.M4_e, lb=-10.0)

model.objective = r_4

In [4]:
model

Name,Blazier et al 2012
Memory address,0x011a596c88
Number of metabolites,7
Number of reactions,10
Objective expression,1.0*R4 - 1.0*R4_reverse_8717c
Compartments,


## Using the data with the model

Let's perform a trivial computation to see how we can integrate the components:

In [5]:
reaction_data = model_expression.to_reaction_dict(condition="Exp#2", model=model)
reaction_data

{'R2': 0.200000000000000,
 'R3': 0.830000000000000,
 'R6': 0.650000000000000,
 'R7': 0.490000000000000}

Here, we see how we are able to map the gene expression data to the respective reaction by using the GPR (Gene-Protein-Reaction) associations.